In [57]:
#Imports
import nltk
import re
import numpy as np
import pickle
from nltk.corpus import stopwords
from sklearn.datasets import load_files

In [58]:
# import the datasets
reviews = load_files('txt_sentoken/')

In [7]:
!ls txt_sentoken        # shows all data(file/folder) in the directory

neg
pos


In [59]:
x, y = reviews.data, reviews.target   # x has data and y has label

In [6]:
y             # 0 for neg and 1 for pos

array([0, 1, 1, ..., 1, 0, 0])

In [8]:
x[0]       # text is in binary form

b"arnold schwarzenegger has been an icon for action enthusiasts , since the late 80's , but lately his films have been very sloppy and the one-liners are getting worse . \nit's hard seeing arnold as mr . freeze in batman and robin , especially when he says tons of ice jokes , but hey he got 15 million , what's it matter to him ? \nonce again arnold has signed to do another expensive blockbuster , that can't compare with the likes of the terminator series , true lies and even eraser . \nin this so called dark thriller , the devil ( gabriel byrne ) has come upon earth , to impregnate a woman ( robin tunney ) which happens every 1000 years , and basically destroy the world , but apparently god has chosen one man , and that one man is jericho cane ( arnold himself ) . \nwith the help of a trusty sidekick ( kevin pollack ) , they will stop at nothing to let the devil take over the world ! \nparts of this are actually so absurd , that they would fit right in with dogma . \nyes , the film is 

In [60]:
re.sub(r'\W', " ", str(x[0]))

'b arnold schwarzenegger has been an icon for action enthusiasts   since the late 80 s   but lately his films have been very sloppy and the one liners are getting worse    nit s hard seeing arnold as mr   freeze in batman and robin   especially when he says tons of ice jokes   but hey he got 15 million   what s it matter to him    nonce again arnold has signed to do another expensive blockbuster   that can t compare with the likes of the terminator series   true lies and even eraser    nin this so called dark thriller   the devil   gabriel byrne   has come upon earth   to impregnate a woman   robin tunney   which happens every 1000 years   and basically destroy the world   but apparently god has chosen one man   and that one man is jericho cane   arnold himself      nwith the help of a trusty sidekick   kevin pollack     they will stop at nothing to let the devil take over the world    nparts of this are actually so absurd   that they would fit right in with dogma    nyes   the film is

In [11]:
re.sub(r'^b\s+', " ", str(x[0]))

'b"arnold schwarzenegger has been an icon for action enthusiasts , since the late 80\'s , but lately his films have been very sloppy and the one-liners are getting worse . \\nit\'s hard seeing arnold as mr . freeze in batman and robin , especially when he says tons of ice jokes , but hey he got 15 million , what\'s it matter to him ? \\nonce again arnold has signed to do another expensive blockbuster , that can\'t compare with the likes of the terminator series , true lies and even eraser . \\nin this so called dark thriller , the devil ( gabriel byrne ) has come upon earth , to impregnate a woman ( robin tunney ) which happens every 1000 years , and basically destroy the world , but apparently god has chosen one man , and that one man is jericho cane ( arnold himself ) . \\nwith the help of a trusty sidekick ( kevin pollack ) , they will stop at nothing to let the devil take over the world ! \\nparts of this are actually so absurd , that they would fit right in with dogma . \\nyes , t

In [61]:
corpus = []
for i in range(len(x)):
    review = re.sub(r'\W', " ", str(x[i]))
    review = review.lower()
    review = re.sub(r'\s+[a-z]\s+', " ",review)
    review = re.sub(r'^b\s+', " ",review)
    review = re.sub(r'\s+', " ",review)
    corpus.append(review)

In [14]:
#corpus

### Bag of word model

In [62]:
# creating BOW model
from sklearn.feature_extraction.text import CountVectorizer

In [63]:
vectorizer = CountVectorizer(max_features=2000, max_df=0.6, stop_words=stopwords.words('english'))
x = vectorizer.fit_transform(corpus).toarray()

### TF-IDF model

In [66]:
# tf-idf model
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
x = transformer.fit_transform(x).toarray()

In [78]:
# tf-idf model directly from corpus
#from sklearn.feature_extraction.text import TfidVectorizer
#vectorizer = TfidVectorizer(max_features=2000, max_df=0.6, stop_words=stopwords.words('english'))
#x = vectorizer.fit_transform(corpus).toarray()

### Train-test split

In [67]:
# Creating train and test dataset
from sklearn.model_selection import train_test_split
text_train, text_test, sent_train, sent_test = train_test_split(x, y, test_size = 0.20, random_state=0)

### Model training

In [68]:
# Model training
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(text_train, sent_train)

LogisticRegression()

### Model testing

In [69]:
# Model testing
sent_pred = classifier.predict(text_test)

### Model Evaluation

In [70]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(sent_test,sent_pred)       # confusion matrix

In [71]:
cm

array([[162,  46],
       [ 27, 165]], dtype=int64)

In [72]:
accuracy_score(sent_test,sent_pred)*100        # accuracy score

81.75

# Saving Classifier

In [29]:
with open('classifier.pickle', 'wb') as f:
    pickle.dump(classifier, f)                 # it will dump the pickle object in the current directory

## Testing with the random sample

In [73]:
sample = ["You are a nice person, Have a good life."]

In [74]:
sample = vectorizer.transform(sample).toarray()

In [75]:
sample

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [76]:
sample[0].sum()

4

In [77]:
print(classifier.predict(sample))           # 1 means positive

[1]


## Using the saved file of classifier

In [43]:
with open('classifier.pickle', 'rb') as f:
    clf = pickle.load(f)

In [44]:
print(clf.predict(sample))

[1]
